<a href="https://colab.research.google.com/github/LgLuigigallo/Webscraper/blob/main/scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install lxml
import requests
!pip install Flask
!pip install flask-restful
!pip install flask-ngrok
!pip install gunicorn
from flask import Flask, jsonify, render_template
from flask_restful import Resource, Api
from flask_ngrok import run_with_ngrok
from bs4 import BeautifulSoup as bs
import json
from datetime import date
import pandas as pd
import numpy as np
from flask import Flask, jsonify,render_template
import html
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%mkdir templates -p
%mkdir static/css -p

In [ ]:
from pip._internal.utils.misc import get_installed_distributions
import sys
#import numpy as np # imported to test whether numpy shows up, which it does!

def get_imported_packages():
    p = get_installed_distributions()
    p = {package.key:package.version for package in p}

    imported_modules = set(sys.modules.keys())
    
    imported_modules.remove('pip')

    modules = [(m, p[m]) for m in imported_modules if p.get(m, False)]

    return modules


def generate_requirements(filepath:str, modules):
    with open(filepath, 'w') as f:
        for module, version in modules:
            f.write(f"{module}=={version}")


generate_requirements('requirements.txt', get_imported_packages())

In [ ]:
# #geographical search
regione="Lazio"
provincia= "Viterbo"
professione= "idraulico"

In [ ]:
#building link

link= "https://www.paginegialle.it/ricerca/" + professione + "/" + provincia + "p-1?mr=200&rk=1"
r = requests.get("https://www.paginegialle.it/ricerca/" + professione + "/" + provincia + "p-2?mr=200&rk=1")
content = r.text
soup = bs(content, "lxml")
link

'https://www.paginegialle.it/ricerca/idraulico/Viterbop-1?mr=200&rk=1'

In [ ]:
#number of results
results=soup.find("span", class_="results-info__numresults")
res = results.text.splitlines()
n_results = int(res[1].lstrip().rstrip())
n_results

70

In [ ]:
#contatore risultati

counter_res=0

dat=pd.DataFrame(columns=['Name', 'Cathegory', 'Phone','Address'])
# ciclo inizializzato per un massimo di 6 pagine con 80 risultati l-una
try:

  for page in range(0,6):
      page=page+1
      # link generation
      link= "https://www.paginegialle.it/ricerca/" + professione + "/" + provincia + "/p-"+str(page)+"?mr=80&rk=1"
      # print(link)
      r = requests.get("https://www.paginegialle.it/ricerca/" + professione + "/" + provincia + "/p-"+str(page)+"?mr=80&rk=1")
      print(link)
      content = r.text
      soup = bs(content, "lxml")
      # soup extraction
      name = soup.find_all("h2", {"class":"list-element__title text-medium ", "itemprop":"name"})
      addr_street = soup.find_all("span", {"itemprop":"streetAddress"})
      addr_loc = soup.find_all("span", {"itemprop":"addressLocality"})
      addr_sugg= soup.find_all("span",  { "itemprop":"address"})
      phone_sugg=soup.find_all("div", {"class":"btn__label"}) 
      phone = soup.find_all("div", {"class":"btn__label", "itemprop":"telephone"})
      categ = soup.find_all("div",{"class":"list-element__category text-medium color-gray-4 mb-10"})
      # result extraction

      for i in range(0,len(name)-1):
          
          counter_res+=1
         
          # deleting suggested result (paid)
          if  (name[i].find("a")) == (name[i].find("a", {"data-pag":"Grande Cliente/RGS"})) :
              print('*****************************************Name suggested removed************************************ \n' )
              phone.insert(i,'000' )
              name.remove(name[i])

          if (name[i].find("a")) != (name[i].find("a", {"data-pag":"Grande Cliente/RGS"})) :
            # print(i)  
            try: 
              print(name[i].find("a").text)
            except:
              name[i]='NaN'
              print("No name")
            try:
              print(categ[i].text.rstrip().lstrip())
            except:
              categ[i]='NaN'
              print("No categories")
            try:
              print(phone[i].text)
            except:
              phone[i]='NaN'
              print("No number")
            try: 
                  print((addr_street[i].text) + ' '+ (addr_loc[i].text)+"\n")        
            except: 
                  addr_street[i]='NaN'
                  addr_loc[i]='Nan'
                  print('No address info')
          try:  
            dat = dat.append([{'Name':name[i].find("a").text, 'Cathegory':categ[i].text.rstrip().lstrip(),'Phone':phone[i].text,
                             'Address':((addr_street[i].text) + ' '+ (addr_loc[i].text))  }], ignore_index=True)
          except Exception as e:
            print(e)
      
except Exception as e: 
  print(e)
            
  print("\n Errore,","Risultati trovati", counter_res,"totale risultati",n_results,sep=' ')


with open('dat.json', 'w') as f:
  f.write(dat.to_json())


print("\n Risultati", counter_res,"totale risultati",n_results, "Risultati omessi",n_results-counter_res ,sep=' ')

https://www.paginegialle.it/ricerca/idraulico/Viterbo/p-1?mr=80&rk=1
Termoidraulica Burioni
IMPIANTI IDRAULICI E TERMOIDRAULICI
0761 345283
Viale Bruno Buozzi, 19 Viterbo

Alma Srls
IMPIANTI IDRAULICI E TERMOIDRAULICI
349 0620332
15/D, Viale Baracca Francesco Viterbo

Termoidraulica di Federico Rossi
IMPRESE EDILI
0761 325909
19, Via S. Pietro Viterbo

Ce.Mar Snc di Cevolo Gianluca e Marcomigni Roberto
IMPIANTI IDRAULICI E TERMOIDRAULICI
328 5387037
Via Galileo Galilei, 1 Viterbo

Fratelli Sorrentino
IMPIANTI IDRAULICI E TERMOIDRAULICI
0761 220625
39/B, Via Belluno Viterbo

La Idrotermica S.r.l.
IMPIANTI IDRAULICI E TERMOIDRAULICI
328 8730294
1, Via Di Pianoscarano Viterbo

Termoidraulica di Stella Sergio & C. S.n.c.
IMPIANTI IDRAULICI E TERMOIDRAULICI
0761 228228
50, Viale Buozzi Bruno Viterbo

Selvaggini Angelo
IMPIANTI IDRAULICI E TERMOIDRAULICI
360 506181
Snc, Via Cava Di Gorga Viterbo

Group Itec S.r.l.
IMPIANTI IDRAULICI E TERMOIDRAULICI
0761 251609
20, Strada Riello Viterbo

Fab

In [ ]:

app=Flask(__name__,template_folder='/content/drive/MyDrive/templates')
run_with_ngrok(app)
with open('/content/dat.json', 'r') as myfile:
      data = myfile.read()


@app.route('/', methods=("POST", "GET"))
def html_table():

    return render_template('simple.html',  tables=[dat.to_html(classes='data')], header="true")

app.run()



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://efdb-35-245-104-168.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Sep/2021 08:17:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2021 08:17:21] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
!pip install pyngrok

!ngrok authtoken xxxxxx
from pyngrok import ngrok
public_url = ngrok.connect(port = '5000')

public_url

  Using cached pyngrok-5.1.0.tar.gz (745 kB)
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=5de837ceb64bd3df6271c87cf01f5241c7b355e0b0a57ca9205c3fd0c79c36e2
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/pyngrok/ngrok.py", line 501, in main
    run(sys.argv[1:])
  File "/usr/local/lib/python3.7/dist-packages/pyngrok/ngrok.py", line 489, in run
    process.run_process(pyngrok_config.ngrok_path, args)
  File "/usr/local/lib/python3.7/dist-packages/pyngrok/process.py", line 339, in run_process
    subprocess.call(start)
  File "/usr/lib/python3.7/subprocess.py", line 341, in call
    return p.wait(timeout=timeout)
  File "/usr/lib/python3.7/subprocess.py", line 1019, in wait
    return self._wait(timeout=timeout)

<NgrokTunnel: "http://3b1f-35-245-104-168.ngrok.io" -> "http://localhost:80">

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
